In [ ]:
# | default_exp downloading

# Downloading
> Fastkafka app for Downloading infobip stuff

In [ ]:
# | export
from os import environ
from pathlib import Path

from airt.infobip.kafka_downloading import create_app

In [ ]:
# | export
root_path = Path(environ["ROOT_PATH"])
downloading_group_id = environ["DOWNLOADING_GROUP_ID"]
# training_group_id = environ["TRAINING_GROUP_ID"]

In [ ]:
# | export
app = create_app(
    root_path=root_path / downloading_group_id, group_id=downloading_group_id
)
